In [ ]:
from data_prepper.data_prepper.data_loader import DataLoader
from data_prepper.data_prepper.data_cleaner import DataCleaner
from data_prepper.data_prepper.feature_extractor import FeatureExtractor
from model_server.model_server.model import Model

In [14]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='data_prepper/data_prepper/.env')

True

In [18]:
data_loader = DataLoader(entsoe_api_key=os.getenv('ENTSOE_API_KEY'))
data_loader.update_df(out_df_filepath="data/bronze/df.parquet")

In [19]:
DataCleaner.clean(
    in_df_filepath='data/bronze/df.parquet',
    out_df_filepath='data/silver/df.parquet',
)

In [ ]:
FeatureExtractor.extract_features(
    in_df_filepath='data/silver/df.parquet',
    out_df_filepath='data/gold/df.parquet',
)

In [ ]:
model = Model(model_filepath='data/model.joblib')
model.train(Xy_filepath='data/gold/df.parquet', n_estimators=100)

In [ ]:
_, mape = model.backtest(
    Xy_filepath='data/gold/df.parquet',
    starting_ts=pd.Timestamp(datetime.now() - pd.Timedelta(30, 'd'), tz='Europe/Zurich'),
    use_every_nth_ts=100,
)
print(f'Backtested MAPE: {mape:.2f}%')

In [10]:
reg = lgb.LGBMRegressor(n_estimators=100, force_row_wise=True, verbose=0)
results_df, mape = ModelTrainer.backtest(
    Xy_filepath='data/gold/df.parquet',
    model=reg,
    starting_ts=pd.Timestamp(datetime.now() - pd.Timedelta(30, 'd'), tz='Europe/Zurich'),
    use_every_nth_ts=10,
)
joblib.dump(reg, 'model.joblib')
print(f'Backtested MAPE: {mape:.2f}%')

100%|███████████████████████████████████████████| 70/70 [00:19<00:00,  3.59it/s]

Backtested MAPE: 5.91%


In [11]:
inferencer = Inferencer(model_filepath='model.joblib')
yhat = inferencer.predict(in_df_filepath='data/gold/df.parquet')
yhat.head(3)

,predicted_24h_later_load
2024-09-27 11:00:00+02:00,6914.421935
2024-09-27 12:00:00+02:00,7051.178301
2024-09-27 13:00:00+02:00,6988.637273
